# Single Lat Long to GeoTiff CRS

In [6]:
def convert_point(lat,long):
    """Convert Coordinate Reference systems from map lat/long
    to fire raster CRS
    Input: Point of latitude,longitude - in that order
    Output: Tuple of (x,y) in CRS coordinates
    """
    
    import geopandas as gpd
    from shapely.geometry import Point
    
    # Declare point (longitude (or x) always comes first, then latitude (y))
    point = Point(long,lat)
    
    # Set Source CRS (Mapbox or Google Maps)
    src_crs = "EPSG:4326"
    
    # create dataframe from input lat/long
    gdf=gpd.GeoDataFrame(index=[0],crs = src_crs, geometry=[point])
        
    # Change CRS to match Wildfire CRS (3857)
    gdf_tf = gdf.to_crs("epsg:3857")
    
    # pull x and y value out from the POINT attribute, then get
    # the value in the data series at row[0]
    x = gdf_tf.geometry.x.at[0]
    y = gdf_tf.geometry.y.at[0]
        
    return (x,y)

In [8]:
x,y = convert_point(36.8,-122.0)
print(x,y)

-13580977.876779376 4411265.910262686


# List of Fire polygon points in Fire CRS to Lat Long

In [3]:
def convert_polygon(fire_polygon):
    """Convert Coord ref system (CRS) from fire data (EPSG 3857) to map 
    lat/long (EPSG 4326), polygon starts and ends on same point (to close it)
    
    Input: list containing tuples of x,y points in fire CRS that describe a polygon
    Output: list containing tuples of lat/long points that describe the polygon
    """
    from shapely import geometry
    import geopandas as gpd
    import numpy as np
    # create polygon
    poly = geometry.Polygon([(p[0], p[1]) for p in fire_polygon])
    
    #CRS
    src_crs = "EPSG:3857"
    dst_crs = "EPSG:4326"
    
    # Create Geo DataFrame
    gfp = gpd.GeoDataFrame(index=[0],crs=src_crs,geometry=[poly])
    
    # Convert CRS
    gfp2 = gfp.to_crs(dst_crs)
    
    # pull data from dataframe
    polyout = gfp2.iloc[0]['geometry']
    
    # create list of tuples, but longitude is first
    l = list(map(tuple,np.asarray(polyout.exterior.coords)))
    l = list(map(lambda m: (m[1],m[0]), l))
    
    return l
    

In [4]:
points = [(-13580977.876779376, 4411265.910262686),
 (-13348542.780003022, 4411265.910262686),
 (-13348542.780003022, 4172037.896749009),
 (-13580977.876779376, 4172037.896749009),
 (-13580977.876779376, 4411265.910262686)]

In [5]:
x = convert_polygon(points)
print(x)

[(36.79999999999999, -122.0), (36.79999999999999, -119.91200000000002), (35.06000000000001, -119.91200000000002), (35.06000000000001, -122.0), (36.79999999999999, -122.0)]


# Convert from Pixels to Raster

In [ ]:
def getcoords(geotif,pixelrow,pixelcol):
    """Get the coordinates of a given pixel in the tif coordinate
    system
    Input: geotiff path+filename, and row,column of the pixel in question
    Output: tuple of the x,y coordinate in the geotif coordinate system
    """
    
    import rasterio as rio
       
    with rio.open(geotif) as tif:
        profile = tif.profile

    left = profile['transform'][2]
    top = profile['transform'][5]
    
    xres = prof['transform'][0]
    yres = prof['transform'][4]
    
    deltax = xres*pixelcol
    deltay = yres*pixelrow
    
    x = left+deltax
    y = top+deltay
    
    return (x,y)

# Convert from Raster to Pixels

In [1]:
def getrasterdims(geotif_filename):
    """Get the top left and resolution for pixels in an array
    given the tif coordinate reference system
    Input: geotiff path+filename
    Output: tuple of the x,y coordinate of top left and resolutions of pixel
    
    Example to find out which pixel a given x,y coordinate refers to:
    pixel_column = (x-left)/xres
    pixel_row = (y-top)/yres
    """
    
    import rasterio as rio
    
    with rio.open(geotif_filename) as tif:
        profile = tif.profile

    left = profile['transform'][2]
    top = profile['transform'][5]
    
    xres = prof['transform'][0]
    yres = prof['transform'][4]
        
    return ((left,top),(xres,yres))

# Lat Long Polygon convert to Fire CRS (EPSG 3857)

In [6]:
def poly_to_fire(longs,lats):
    """Convert the longitude/latitude polygon from Mapbox input
    to Fire CRS (EPSG 3857)
    Input: List of Longitudes and Latitudes that form a polygon
    Output: List of (X,Y) tuples defining polygon on the fire CRS
    """
    
    import geopandas as gpd
    import rasterio as rio
    from shapely.geometry import Polygon
    # Define CRSs
    poly_crs = "epsg:4326" # Long/Lat from map
    fire_crs = "epsg:3857"
    
    # Create polygon and put in GDF
    poly_poly = Polygon(zip(longs,lats))
    poly_gdf = gpd.GeoDataFrame(index=[0],crs=poly_crs,geometry = [poly_poly])
    
    #Transform the gdf crs
    outgdf = poly_gdf.to_crs(fire_crs)
    
    # pull data from dataframe
    polyout = outgdf.iloc[0]['geometry']
    
    # create list of tuples of x,y
    l = list(map(tuple,np.asarray(polyout.exterior.coords)))
    
    return l
    
    